<a href="https://colab.research.google.com/github/anilbhatt1/CNN-EIP4-S5/blob/master/EIP4_S5_Assignment_5_PersonAttrubutes_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

In [0]:
%tensorflow_version 1.x

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator


import numpy as np
import time, math
from tqdm import tqdm_notebook as tqdm  #This is for progress bar
import matplotlib.pyplot as plt
#from random_eraser import get_random_eraser

import tensorflow as tf
#from tensorflow.keras import Sequence
from tensorflow.python.keras.utils.data_utils import Sequence

In [0]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

In [0]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

In [0]:
import keras
import numpy as np
from tensorflow.python.keras.utils.data_utils import Sequence

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonalDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=32, shuffle=True):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
       # self.df.shape[0] = shape

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)


In [0]:
one_hot_df

In [0]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15)
train_df.shape, val_df.shape

In [0]:
train_df.head()

In [0]:
# create train and validation data generators
train_gen = PersonalDataGenerator(train_df, batch_size=32)
valid_gen = PersonalDataGenerator(val_df, batch_size=64, shuffle=False)

In [0]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

In [0]:
len(num_units)

In [0]:
class Conv_bottleNeck(tf.keras.Model):
    expansion = 4
    def __init__(self,input_channels,channels,stride=1,dim_change=None):
        super(Conv_bottleNeck,self).__init__()
        
        self.conv1 = tf.keras.layers.Conv2D(filters=channels,kernel_size=1,strides=1)  # 1x1xdesired channels
        self.bn1 = tf.keras.layers.BatchNormalization() # Batch-Normalization
        self.conv2 = tf.keras.layers.Conv2D(filters=channels,kernel_size=3,strides=stride,padding='same') # 3x3xdesired channels
        self.bn2 = tf.keras.layers.BatchNormalization() # Batch-Norm
        self.conv3 = tf.keras.layers.Conv2D(filters=channels*self.expansion,kernel_size=1)  # 1x1x4*desired channels that were at begining
        self.bn3 = tf.keras.layers.BatchNormalization() # Batch-norm
        self.dim_change = dim_change # Will get this dim_change from Resnet class
    
    def forward(self,x):    # Main logic 
        res = x
        
        output = tf.nn.relu(self.bn1(self.conv1(x)))  # 1x1xinput size -> BN -> Relu
        output = tf.nn.relu(self.bn2(self.conv2(output)))  # 3x3xinput size -> BN -> Relu
        output = self.bn3(self.conv3(output))  # 3x3x4*input size -> BN

        if self.dim_change is not None:
            res = self.dim_change(res)   # Applying dim_change we get from ResNet class to the residual block if dimension expansion is there
        
        output += res             # Adding residual block with skip network
        output = tf.nn.relu(output)   # Applying relu on combined output
        return output


In [0]:
class Resnet_50(tf.keras.Model):

    def __init__(self,block_type,num_layers,classes=8):
        super(Resnet_50,self).__init__()
        
        self.inp_channels=64
        self.conv1_res = tf.keras.layers.Conv2D(filters=self.inp_channels, kernel_size=7, strides=2,padding='same',use_bias=False)
        self.bn1_res = tf.keras.layers.BatchNormalization() # There needs to have a Relu after this
        self.pool2D_res = tf.keras.layers.MaxPooling2D()
        self.layer1 = self._layer(block_type,64,num_layers[0],stride=1)
        self.layer2 = self._layer(block_type,128,num_layers[1],stride=2) 
        self.layer3 = self._layer(block_type,256,num_layers[2],stride=2) 
        self.layer4 = self._layer(block_type,512,num_layers[3],stride=2)
        self.avgpool = tf.keras.layers.GlobalAveragePooling2D()
        self.fully_connected = tf.keras.layers.Dense(8, kernel_initializer='he_normal', use_bias=False)

    def _layer(self,block_type,channels,numb_layers,stride=1):
         dim_change = None
         if stride!=1 or channels != self.inp_channels*block_type.expansion:
             dim_change = tf.keras.Sequential(tf.keras.layers.Conv2D(filters=channels*block_type.expansion,kernel_size=1,strides=stride),
                                             tf.keras.layers.BatchNormalization())
         netLayers =[]
         netLayers.append(block_type(self.inp_channels,channels,stride=stride,dim_change=dim_change))
         self.inp_channels = channels * block_type.expansion
         for i in range(1,numb_layers):
           netLayers.append(block_type(self.inp_channels,channels))
           self.inp_channels = channels * block_type.expansion
         return tf.keras.Sequential(netLayers)

    def forward(self,x):    # Main logic
      x = tf.nn.relu(self.bn1_res(self.conv1_res(x)))  # 7x7 -> BN -> Relu
      x = self.pool2D_res(x)   # Maxpooling
      x = self.layer1(x)       # Logic to build layer 1, for Resnet 50 there will be 3 blocks
      x = self.layer2(x)       # Building layer 2, there will be 4 blocks
      x = self.layer3(x)       # Building layer 3, there will be 6 blocks
      x = self.layer4(x)       # Building layer 4, there will be 3 blocks
      x = self.avgpool(x)      # Global Average pooling
      x = self.fully_connected(x)  # Fully connected

In [0]:
model = Resnet_50(Conv_bottleNeck,[3,4,6,3])

In [0]:
# losses = {
# 	"gender_output": "binary_crossentropy",
# 	"image_quality_output": "categorical_crossentropy",
# 	"age_output": "categorical_crossentropy",
# 	"weight_output": "categorical_crossentropy",

# }
# loss_weights = {"gender_output": 1.0, "image_quality_output": 1.0, "age_output": 1.0}
opt = SGD(lr=0.001, momentum=0.9)
model.compile(
    optimizer='adam',
    loss="categorical_crossentropy", 
    # loss_weights=loss_weights, 
    metrics=['accuracy'])

In [0]:
# model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=32, epochs=10)

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=10,
    verbose=1
)

In [0]:
#from keras.preprocessing.image import ImageDataGenerator

#datagen = ImageDataGenerator(zoom_range=0.0, 
                             horizontal_flip=False)
# train the model
#start = time.time()

#model_info = model.fit_generator(datagen.flow(train_gen, batch_size = 128),
#                                 samples_per_epoch = train_gen.shape[0], nb_epoch = 10, 
#                                 validation_data = (val_gen), verbose=1)

In [0]:
model